Тут находится мой бейзлайн, адаптированный для английского языка. 

Как запустить:
1. установить requirements.txt
2. скачать английский wordnet и положить в папку data/wordnet_data
3. возможно, скачать какой-то ваш любимый английский word2vec и указать правильный путь до него в ячейках в следующем разделе
4. параметрами кодирования текстов (например, весами POS и добавлением POS тегов при поиске слов) можно управлять в ячейках, где создаётся W2VWrapper

# load data

In [3]:
import os
import pandas as pd
import numpy as np
import json

In [4]:
from nltk.corpus import WordNetCorpusReader

In [58]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/dale/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/dale/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/dale/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [43]:
from sklearn.neighbors import KDTree

In [37]:
import gensim.downloader

In [ ]:
import xmltodict
import numpy as np
import pandas as pd

from functools import lru_cache
from collections import Counter, defaultdict
from tqdm.auto import tqdm

In [119]:
existing_taxonomy = WordNetCorpusReader(os.path.join('data', 'WN2.0'), None)

In [238]:
nouns_data = pd.read_csv(os.path.join('data', 'wordnet_data', 'nouns_en_new.2.0-3.0.tsv'), sep='\t', header=None)
nouns_data.columns = ['word', 'hypernyms']
nouns_data['hypernyms'] = nouns_data['hypernyms'].apply(json.loads)
print(nouns_data.shape)

(3129, 2)


In [239]:
nouns_data.sample(10)

,word,hypernyms
969,Greco-Roman_wrestling,"[wrestling.n.02, contact_sport.n.01]"
2901,Rastafarian,"[disciple.n.01, follower.n.01]"
772,core_memory,"[random-access_memory.n.01, volatile_storage.n..."
1735,Succos,"[festival.n.01, religious_festival.n.01]"
1089,neoconservatism,"[conservatism.n.01, political_orientation.n.01]"
1563,Raja_batis,"[ray.n.07, skate.n.02]"
1662,coronary_heart_disease,"[cardiovascular_disease.n.01, heart_disease.n.01]"
279,Petersburg,"[operation.n.05, campaign.n.03]"
616,historiography,"[writing.n.02, literature.n.03]"
2975,rawness,"[ignorance.n.01, content.n.05]"


In [240]:
verbs_data = pd.read_csv(os.path.join('data', 'wordnet_data', 'verbs_en_new.2.0-3.0.tsv'), sep='\t', header=None)
verbs_data.columns = ['word', 'hypernyms']
verbs_data['hypernyms'] = verbs_data['hypernyms'].apply(json.loads)
print(verbs_data.shape)
verbs_data.sample(10)

(207, 2)


,word,hypernyms
125,pumice,"[guide.v.05, rub.v.01]"
51,currycomb,"[comb.v.01, straighten.v.02]"
153,degrease,[get_rid_of.v.01]
205,cutinize,"[change.v.01, convert.v.02]"
83,nol.pros.,"[discontinue.v.01, drop.v.07]"
42,devein,[get_rid_of.v.01]
18,air-ship,"[transport.v.01, freight.v.01]"
183,sideline,"[demote.v.01, delegate.v.02]"
174,metalize,"[cover.v.01, coat.v.01]"
114,yak,"[communicate.v.02, interact.v.01]"


In [172]:
import evaluate # from https://github.com/dialogue-evaluation/taxonomy-enrichment

In [251]:
morph_parse("get")

'VB'

In [252]:
tokenize('air-ship')

['air-ship']

# Prepare dependencies

In [307]:
!wget http://vectors.nlpl.eu/repository/20/29.zip  # http://vectors.nlpl.eu/explore/embeddings/en/models/ gigaword

--2020-11-02 15:50:18--  http://vectors.nlpl.eu/repository/20/29.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 667161074 (636M) [application/zip]
Saving to: ‘29.zip’

29.zip              100%[===================>] 636,25M  20,3MB/s    in 2m 13s  

2020-11-02 15:52:31 (4,80 MB/s) - ‘29.zip’ saved [667161074/667161074]



In [311]:
!unzip 29.zip -d gigaword

Archive:  29.zip
  inflating: gigaword/meta.json      
  inflating: gigaword/model.bin      
  inflating: gigaword/model.txt      
  inflating: gigaword/README         


In [315]:
gw_vecs = gensim.models.KeyedVectors.load_word2vec_format('gigaword/model.bin', binary=True)

In [324]:
gw_vecs.most_similar('google_VERB')

[('mispronounce_VERB', 0.4861705005168915),
 ('type_VERB', 0.4858720600605011),
 ('typing_NOUN', 0.4777812957763672),
 ('bookmark_VERB', 0.47726932168006897),
 ('spell-check_NOUN', 0.4647689461708069),
 ('facebook_NOUN', 0.46326524019241333),
 ('retype_VERB', 0.4609655737876892),
 ('username_ADJ', 0.45739173889160156),
 ('mistype_VERB', 0.452288419008255),
 ('login_NOUN', 0.4498198628425598)]

In [ ]:
google_vecs = gensim.downloader.load('word2vec-google-news-300')

In [56]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 

In [147]:
@lru_cache(maxsize=50_000)
def lemmatize(word):
    return lemmatizer.lemmatize(word) or word

In [167]:
@lru_cache(maxsize=50_000)
def morph_parse(w):
    return nltk.pos_tag([w])[0][1]

tokenize = nltk.word_tokenize
word2pos  = morph_parse

In [63]:
def normalize(v, epsilon=1e-10):
    return v / (sum(v**2)**0.5 + epsilon)

In [149]:
class BaseSentenceEmbedder:
    def __init__(self, n=300, normalize_word=True, pos_weights=None, default_weight=1):
        self.n = n
        self.normalize_word = normalize_word
        self.pos_weights = pos_weights
        self.default_weight = default_weight

    def get_word_vec(self, word):
        raise NotImplementedError()

    @lru_cache(maxsize=1024)
    def __call__(self, text):
        tokens = tokenize(text)
        weights = self.get_word_weights(tokens)
        vecs = [self.get_word_vec(word) for word in tokens]
        if self.normalize_word:
            vecs = [normalize(v) for v in vecs]
        if len(vecs) == 0:
            return np.zeros(self.n)
        return normalize(sum([vec * weight for vec, weight in zip(vecs, weights)]))

    def get_word_weights(self, tokens):
        if not self.pos_weights:
            return [1] * len(tokens)
        return [self.pos_weights.get(word2pos(t), self.default_weight) for t in tokens]

In [375]:
class W2VWrapper(BaseSentenceEmbedder):
    POS_MAP = {
        'INFN': 'VERB',
        'ADJF': 'ADJ',
        'ADVB': 'ADV',
    }
    POS_MISS = {
        'PREP'
    }

    def __init__(self, w2v, morph=morph_parse, append_pos=False, **kwargs):
        super(W2VWrapper, self).__init__(**kwargs)
        self.w2v = w2v
        self.morph = morph
        self.prefix2word = self.make_prefixes()
        self.append_pos = append_pos

    def make_prefixes(self):
        prefix2word = defaultdict(set)
        for w in tqdm(self.w2v.vocab.keys()):
            for n in range(1, len(w) - 2):
                prefix2word[w[:-n]].add(w)
        prefix2word = {k: v for k, v in prefix2word.items()}
        return prefix2word

    def find_prefix(self, word, min_len=2):
        mapping = self.prefix2word
        if word in mapping:
            return mapping[word]
        for i in range(1, len(word) - min_len):
            t = mapping.get(word[:-i])
            if t is not None:
                return t
        return None

    def get_text_vec(self, text, verbose=False):
        toks = tokenize(text)
        vecs = []
        for tok in toks:
            vecs.append(self.get_word_vec(word=tok, verbose=verbose))
        if not vecs:
            return np.zeros(self.w2v.vectors.shape[1])
        return normalize(sum(vecs))

    def add_pos(self, word):
        tag = self.morph(word)
        if not tag:
            return word
        new_pos = self.POS_MAP.get(tag, tag or '-')
        return word + '_' + new_pos
    
    def get_word_vec(self, word, verbose=False, add_pos=False):
        key = word
        key2 = lemmatize(word)
        if self.append_pos:
            key = self.add_pos(key)
            key2 = self.add_pos(key2)
        if verbose:
            print(key, key2, self.find_prefix(key))
        if key in self.w2v.vocab:
            return self.w2v[key]
        elif key2 in self.w2v.vocab:
            return self.w2v[key2]
        else:
            keys = self.find_prefix(key)
            if keys:
                return sum([self.w2v[k] for k in keys]) / len(keys)
        return np.zeros(self.n)


In [376]:
w2v_embedder_gw = W2VWrapper(gw_vecs, append_pos=True)

In [381]:
POS_MAP = {
    #CC coordinating conjunction
    'CD': 'NUM', # cardinal digit
    #DT determiner
    #EX existential there (like: “there is” … think of it like “there exists”)
    'FW': 'X', # foreign word
    #IN preposition/subordinating conjunction
    'JJ': 'ADJ', #adjective ‘big’
    'JJR': 'ADJ', #adjective, comparative ‘bigger’
    'JJS': 'ADJ', #adjective, superlative ‘biggest’
    'LS': 'NUM', #list marker 1)
    'MD': 'VERB',# modal could, will
    'NN': 'NOUN', #noun, singular ‘desk’
    'NNS': 'NOUN', #noun plural ‘desks’
    'NNP': 'PROPN', #proper noun, singular ‘Harrison’
    'NNPS': 'PROPN', #proper noun, plural ‘Americans’
    #PDT predeterminer ‘all the kids’
    'POS': 'NOUN', #possessive ending parent’s
    #'PRP personal pronoun I, he, she
    # PRP$ possessive pronoun my, his, hers
    'RB': 'ADV', #adverb very, silently,
    'RBR': 'ADV', #adverb, comparative better
    'RBS': 'ADV', #adverb, superlative best
    #RP particle give up
    #TO, to go ‘to’ the store.
    #UH interjection, errrrrrrrm
    'VB': 'VERB', #verb, base form take
    'VBD': 'VERB',# verb, past tense, took
    'VBG': 'VERB', #verb, gerund/present participle taking
    'VBN': 'VERB', #verb, past participle is taken
    'VBP': 'VERB', #verb, sing. present, known-3d take
    'VBZ': 'VERB', #verb, 3rd person sing. present takes
    #WDT wh-determiner which
    #WP wh-pronoun who, what
    #WP$ possessive wh-pronoun whose
    #WRB wh-adverb where, when
}

In [382]:
w2v_embedder_gw.POS_MAP = POS_MAP

In [383]:
w2v_embedder_gw_pos_n = W2VWrapper(gw_vecs,  pos_weights={'NN': 1.0, 'NNS': 1, 'NNP': 1, 'NNPS':1, 'IN': 0.1, 'CC': 0.1}, default_weight=0.5)
w2v_embedder_gw_pos_v = W2VWrapper(gw_vecs,  pos_weights={'VB': 1.0, 'VBG': 1, 'IN': 0.1, 'CC': 0.1} , default_weight=0.5)
w2v_embedder_gw_pos_n.POS_MAP = POS_MAP
w2v_embedder_gw_pos_v.POS_MAP = POS_MAP

In [265]:
w2v_embedder = W2VWrapper(google_vecs)

In [266]:
w2v_embedder_pos_n = W2VWrapper(google_vecs,  pos_weights={'NN': 1.0, 'IN': 0.1, 'CC': 0.1}, default_weight=0.5)

Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x7ff478044440>
Traceback (most recent call last):
  File "/home/dale/anaconda3/lib/python3.7/_weakrefset.py", line 38, in _remove
    def _remove(item, selfref=ref(self)):
KeyboardInterrupt


In [277]:
w2v_embedder_pos_v = W2VWrapper(google_vecs,  pos_weights={'VB': 1.0, 'IN': 0.1, 'CC': 0.1}, default_weight=0.5)

In [157]:
class SynsetStorage:
    def __init__(self, id2synset, ids, ids_long, texts_long, word2sense, forbidden_id=None):
        self.id2synset = id2synset
        self.ids = ids
        self.ids_long = ids_long
        self.texts_long = texts_long
        self.word2sense = word2sense
        self.forbidden_id = forbidden_id or set()

 
    @classmethod
    def from_taxonomy(cls, tx, pos, forbidden_words=None):
        id2synset = {v.name(): v for v in tx.all_synsets(pos=pos)}
        print('synsets: {}'.format(len(id2synset)))
        
        word2sense = defaultdict(set)
        for synset_id, synset in id2synset.items():
            texts = {l.name() for l in synset.lemmas()}
            texts.add(synset.name().split('.')[0])
            # texts.add(synset.definition())
            for text in texts:
                word2sense[text].add(synset_id)
        print('senses', len(word2sense))
        
        forbidden_id = set()
        for word in (forbidden_words or {}):
            if word in word2sense:
                for sense_id in word2sense[word]:
                    forbidden_id.add(sense_id)
        print('forbidden senses are', len(forbidden_id))
        
        ids = sorted(id2synset.keys())
        ids_long = []
        texts_long = []
        for id in tqdm(ids):
            synset = id2synset[id]

            texts = {l.name() for l in synset.lemmas()}
            texts.add(synset.name().split('.')[0])
            texts.add(synset.definition())

            # исключаем все слова, омонимичные с тем, что есть в тестовой выборке
            senses = {synset_id for w in texts for synset_id in word2sense[w]}
            if senses.intersection(forbidden_id):
                continue

            if len(texts) > 1:
                texts.add(' ; '.join(sorted(texts)))
            for text in sorted(texts):
                ids_long.append(id)
                texts_long.append(text)
        
        print('numer of ids', len(ids), 'long list is', len(ids_long))
        return cls(
            id2synset=id2synset,
            ids=ids,
            ids_long=ids_long,
            texts_long=texts_long,
            word2sense=word2sense,
            forbidden_id=forbidden_id,
        )

    def get_synset_name(self, synset_id):
        return self.id2synset.get(synset_id, {}).get('@ruthes_name', '-')


def make_rel_df(rel_n_raw, id2synset):
    rel_df = pd.DataFrame(rel_n_raw['relations']['relation'])
    rel_df['parent'] = rel_df['@parent_id'].apply(lambda x: id2synset[x]['@ruthes_name'])
    rel_df['child'] = rel_df['@child_id'].apply(lambda x: id2synset.get(x, {}).get('@ruthes_name'))
    return rel_df


class RelationStorage:
    def __init__(self, forbidden_id=None):
        self.id2hyponym = defaultdict(set)
        self.id2hypernym = defaultdict(set)
        self.forbidden_id = forbidden_id or set()  # forbidden_id = set(ttest.SYNSET_ID)

    def add_pair(self, hypo_id, hyper_id, max_depth=100500):
        if max_depth <= 0:
            return
        if hypo_id in self.id2hyponym[hyper_id]:
            # the pair is already here
            return
        if hypo_id in self.id2hypernym[hyper_id]:
            raise ValueError('{} is already a hypernym of {}, so it cannot become its hyponym'.format(hypo_id, hyper_id))
        for next_hypo in self.id2hyponym[hypo_id]:
            self.add_pair(next_hypo, hyper_id, max_depth=max_depth-1)
        for next_hyper in self.id2hypernym[hyper_id]:
            self.add_pair(hypo_id, next_hyper, max_depth=max_depth-1)
        self.id2hyponym[hyper_id].add(hypo_id)
        self.id2hypernym[hypo_id].add(hyper_id)

    def load_relations_from_taxonomy(self, tx, pos):
        self.id2hyponym = defaultdict(set)
        self.id2hypernym = defaultdict(set)
        for v in tx.all_synsets(pos=pos):
            hypo_id = v.name()
            if hypo_id in self.forbidden_id:
                continue
            for hyper in v.hypernyms():
                hyper_id = hyper.name()
                if hyper_id not in self.forbidden_id:
                    self.add_pair(hypo_id, hyper_id, max_depth=1)
        print(len(self.id2hyponym))
        print(max(len(c) for c in self.id2hyponym.values()))
        print(max(len(c) for c in self.id2hypernym.values()))
        print(sum(len(c) for c in self.id2hypernym.values()))
            
        
    def construct_relations(self, rel_df):
        self.id2hyponym = defaultdict(set)
        self.id2hypernym = defaultdict(set)

        hypo_df = rel_df[rel_df['@name'] == 'hyponym']
        for r, row in tqdm(hypo_df.iterrows()):
            hypo_id = row['@child_id']
            hyper_id = row['@parent_id']
            if hypo_id not in self.forbidden_id and hyper_id not in self.forbidden_id:
                self.add_pair(hypo_id, hyper_id, max_depth=1)  # во второй версии поставим максимальную глубину, равную 2

        print(len(self.id2hyponym))
        print(max(len(c) for c in self.id2hyponym.values()))
        print(max(len(c) for c in self.id2hypernym.values()))
        print(sum(len(c) for c in self.id2hypernym.values()))


In [290]:
def hypotheses_knn(
        text,
        synset_storage: SynsetStorage,
        rel_storage: RelationStorage,
        index=None,
        text2vec=None,
        k=10,
        verbose=False,
        decay=0,
        grand_mult=1,
        result_size=10,
        return_hypotheses=False,
        neighbor_scorer=None,
        indexer=None,
        scorer_pow=None,
):
    ids_list = synset_storage.ids_long
    texts_list = synset_storage.texts_long
    # todo: distance decay
    if indexer:
        distances, indices = indexer.query(text, k=k)
    else:
        vec = text2vec(text)
        distances, indices = index.query(vec.reshape(1, -1), k=k)
    hypotheses = Counter()
    for i, d in zip(indices.ravel(), distances.ravel()):
        s = 1 - d**2 / 2
        hypers = rel_storage.id2hypernym.get(ids_list[i], set())

        if neighbor_scorer is not None:
            neighbor_score = neighbor_scorer(text, texts_list[i])
        elif scorer_pow is not None:
            neighbor_score = s**scorer_pow
        else:
            neighbor_score = 1
        base_score = np.exp(-d ** decay) * neighbor_score
        if verbose:
            print(d, 1, ids_list[i], texts_list[i], len(hypers), np.exp(-d ** decay),  base_score)
        for parent in hypers:
            hypotheses[parent] += base_score
            for grandparent in rel_storage.id2hypernym.get(parent, set()):
                hypotheses[grandparent] += base_score * grand_mult
    if return_hypotheses:
        return hypotheses
    if verbose:
        print(len(hypotheses))
    result = []
    for hypo, cnt in hypotheses.most_common(result_size):
        if verbose:
            print(cnt, hypo, synset_storage.id2synset[hypo].name())
        result.append(hypo)
    return result

In [155]:
full_syn_storage = SynsetStorage.from_taxonomy(tx=existing_taxonomy, pos=tx.NOUN)

synsets: 79689
senses 132127
forbidden senses are 0



numer of ids 79689 long list is 319887


In [159]:
rel_storage = RelationStorage(forbidden_id=full_syn_storage.forbidden_id)
rel_storage.load_relations_from_taxonomy(tx=existing_taxonomy, pos=tx.NOUN)

79689
619
6
81857


Embedder is the longest part

In [156]:
full_w2v_vecs = np.stack([w2v_embedder(t) for t in tqdm(full_syn_storage.texts_long) ])
full_w2v_tree = KDTree(full_w2v_vecs)

In [278]:
full_w2v_vecs_pos = np.stack([w2v_embedder_pos(t) for t in tqdm(full_syn_storage.texts_long) ])
full_w2v_tree_pos = KDTree(full_w2v_vecs_pos)

In [175]:
def normalize_input_text(text):
    return text.replace('_', ' ').replace('-', ' ').lower().strip()

In [178]:
train_data['word_norm'] = train_data['word'].apply(normalize_input_text)

In [185]:
smpl = train_data.sample(100, random_state=1)
smpl_gt = {row.word_norm: row.hypernyms for i, row in smpl.iterrows()}
print(*evaluate.get_score(smpl_gt, smpl_gt, k=10))

1.0 0.0


In [284]:

def w2v_scorer(text1, text2):
    return np.dot(w2v_embedder.get_text_vec(text1), w2v_embedder.get_text_vec(text2)) ** 5

In [188]:
public_test_hypos = {
    txt: 
    hypotheses_knn(
        txt, 
        #index=full_ft_tree, text2vec=ft_embedder, 
        index=full_w2v_tree, text2vec=w2v_embedder_pos,
        synset_storage=full_syn_storage, rel_storage=rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        neighbor_scorer=None, # todo: return it
    )  
    for txt in tqdm(smpl.word_norm)
}
print(*evaluate.get_score(smpl_gt, public_test_hypos, k=10))  # 0.1344


0.1344155844155844 0.0


In [189]:
public_test_hypos = {
    txt: 
    hypotheses_knn(
        txt, 
        index=full_w2v_tree, text2vec=w2v_embedder,
        synset_storage=full_syn_storage, rel_storage=rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        neighbor_scorer=None, # todo: return it
    )  
    for txt in tqdm(smpl.word_norm)
}
print(*evaluate.get_score(smpl_gt, public_test_hypos, k=10))  # 0.1338


0.13384439634439632 0.0


In [202]:
public_test_hypos = {
    txt: 
    hypotheses_knn(
        txt, 
        index=full_w2v_tree, text2vec=w2v_embedder,
        synset_storage=full_syn_storage, rel_storage=rel_storage,
        decay=0.03, # ok
        k=100, 
        grand_mult=0.6,
        neighbor_scorer=None, # todo: return it
    )  
    for txt in tqdm(smpl.word_norm)
}
print(*evaluate.get_score(smpl_gt, public_test_hypos, k=10))  # 0.14755491422158085 


0.14755491422158085 0.0


In [204]:
def w2v_scorer(text1, text2):
    return np.dot(w2v_embedder.get_text_vec(text1), w2v_embedder.get_text_vec(text2)) ** 5 

public_test_hypos = {
    txt: 
    hypotheses_knn(
        txt, 
        #index=full_ft_tree, text2vec=ft_embedder, 
        index=full_w2v_tree, text2vec=w2v_embedder_pos,
        synset_storage=full_syn_storage, rel_storage=rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        neighbor_scorer=w2v_scorer, # todo: return it
    )  
    for txt in tqdm(smpl.word_norm)
}
print(*evaluate.get_score(smpl_gt, public_test_hypos, k=10))  # 0.1706088664421998 


0.1706088664421998 0.0


In [214]:
def w2v_scorer(text1, text2):
    return np.dot(w2v_embedder.get_text_vec(text1), w2v_embedder.get_text_vec(text2)) ** 10

public_test_hypos = {
    txt: 
    hypotheses_knn(
        txt, 
        #index=full_ft_tree, text2vec=ft_embedder, 
        index=full_w2v_tree, text2vec=w2v_embedder_pos,
        synset_storage=full_syn_storage, rel_storage=rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.8,
        neighbor_scorer=w2v_scorer, # todo: return it
    )  
    for txt in tqdm(smpl.word_norm)
}
print(*evaluate.get_score(smpl_gt, public_test_hypos, k=10))  # 0.18505892255892262 


0.18505892255892262 0.0


In [230]:
public_test_hypos = {
    txt: 
    hypotheses_knn(
        txt, 
        #index=full_ft_tree, text2vec=ft_embedder, 
        index=full_w2v_tree, text2vec=w2v_embedder_pos,
        synset_storage=full_syn_storage, rel_storage=rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.8,
        scorer_pow=10
    )  
    for txt in tqdm(smpl.word_norm)
}
print(*evaluate.get_score(smpl_gt, public_test_hypos, k=10))  # 0.18505892255892262 


0.18164582331248996 0.0


In [216]:
def dict2submission(word2hypotheses, id2synset):
    result_nouns = []
    result_hyperonyms = []
    for n, h in word2hypotheses.items():
        for hypo in h:
            result_nouns.append(n)
            result_hyperonyms.append(hypo)
    result_df = pd.DataFrame({'noun': result_nouns, 'result': result_hyperonyms})
    return result_df

In [221]:
len(public_test_hypos)

99

In [220]:
dict2submission(public_test_hypos, id2synset).noun.nunique()

99

In [219]:
public_test_hypos

AttributeError: 'dict' object has no attribute 'noun'

# submit for nouns

In [241]:

test_n_hypos = {
    txt: 
    hypotheses_knn(
        normalize_input_text(txt), 
        #index=full_ft_tree, text2vec=ft_embedder, 
        index=full_w2v_tree, text2vec=w2v_embedder_pos,
        synset_storage=full_syn_storage, rel_storage=rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.8,
        scorer_pow=5
    )   
    for txt in tqdm(nouns_data.word)
}

In [244]:
sub = dict2submission(test_n_hypos, full_syn_storage.id2synset)
sub.to_csv('nouns_test_result2.tsv', sep='\t', encoding='utf-8', header=None, index=None)
sub.head(15)

,noun,result
0,ready-made,state.n.04
1,ready-made,condiment.n.01
2,ready-made,artifact.n.01
3,ready-made,buffer.n.03
4,ready-made,spread.n.05
5,ready-made,memory_device.n.01
6,ready-made,cost.n.01
7,ready-made,container.n.01
8,ready-made,payment.n.01
9,ready-made,sheet.n.06


In [245]:
! python evaluate.py data/wordnet_data/nouns_en_new.2.0-3.0.tsv nouns_test_result2.tsv

map: 0.18766891584878223
mrr: 0.20641630316248633



In [285]:

test_n_hypos = {
    txt: 
    hypotheses_knn(
        normalize_input_text(txt), 
        #index=full_ft_tree, text2vec=ft_embedder, 
        index=full_w2v_tree_pos, text2vec=w2v_embedder_pos,
        synset_storage=full_syn_storage, rel_storage=rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        neighbor_scorer=w2v_scorer,
        #scorer_pow=5
    )   
    for txt in tqdm(nouns_data.word)
}
sub = dict2submission(test_n_hypos, full_syn_storage.id2synset)
sub.to_csv('nouns_test_result_tmp.tsv', sep='\t', encoding='utf-8', header=None, index=None)

! python evaluate.py data/wordnet_data/nouns_en_new.2.0-3.0.tsv nouns_test_result_tmp.tsv


map: 0.18766891584878223
mrr: 0.20641630316248633



In [286]:
! python evaluate.py data/wordnet_data/nouns_en_new.2.0-3.0.tsv nouns_test_result_tmp.tsv

map: 0.23411422613998975
mrr: 0.2543461468556893



In [395]:
full_w2v_vecs_n_GW = np.stack([w2v_embedder_gw(t) for t in tqdm(full_syn_storage.texts_long) ])
full_w2v_tree_n_GW = KDTree(full_w2v_vecs_n_GW)

In [ ]:
full_w2v_vecs_gw_pos_n = np.stack([w2v_embedder_gw_pos_n(t) for t in tqdm(full_syn_storage_n.texts_long) ])
full_w2v_tree_gw_pos_n = KDTree(full_w2v_vecs_gw_pos_n)

In [398]:
def w2v_scorer(text1, text2):
    return np.dot(w2v_embedder_gw.get_text_vec(text1), w2v_embedder_gw.get_text_vec(text2)) ** 5

test_v_hypos = {
    txt: 
    hypotheses_knn(
        normalize_input_text(txt), 
        #index=full_ft_tree, text2vec=ft_embedder, 
        index=full_w2v_tree_n_GW, text2vec=w2v_embedder_gw,
        #index=full_w2v_tree_gw_pos_v, text2vec=w2v_embedder_gw_pos_v,
        synset_storage=full_syn_storage, rel_storage=rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        neighbor_scorer=w2v_scorer,
        #scorer_pow=5
    )   
    for txt in tqdm(nouns_data.word)
}
sub = dict2submission(test_n_hypos, full_syn_storage.id2synset)
sub.to_csv('nouns_test_result_tmp.tsv', sep='\t', encoding='utf-8', header=None, index=None)
! python evaluate.py data/wordnet_data/nouns_en_new.2.0-3.0.tsv nouns_test_result_tmp.tsv


map: 0.23411422613998975
mrr: 0.2543461468556893



## submit for verbs

In [234]:
full_syn_storage_v = SynsetStorage.from_taxonomy(tx=existing_taxonomy, pos=tx.VERB)
rel_storage_v = RelationStorage(forbidden_id=full_syn_storage_v.forbidden_id)
rel_storage_v.load_relations_from_taxonomy(tx=existing_taxonomy, pos=tx.VERB)

synsets: 13508
senses 11319
forbidden senses are 0



numer of ids 13508 long list is 51663
13285
393
2
12985


In [235]:
full_w2v_vecs_v = np.stack([w2v_embedder(t) for t in tqdm(full_syn_storage_v.texts_long) ])
full_w2v_tree_v = KDTree(full_w2v_vecs_v)

In [279]:
full_w2v_vecs_v_pos = np.stack([w2v_embedder_pos_v(t) for t in tqdm(full_syn_storage_v.texts_long) ])
full_w2v_tree_v_pos = KDTree(full_w2v_vecs_v_pos)

In [378]:
full_w2v_vecs_v_GW = np.stack([w2v_embedder_gw(t) for t in tqdm(full_syn_storage_v.texts_long) ])
full_w2v_tree_v_GW = KDTree(full_w2v_vecs_v_GW)

In [384]:
full_w2v_vecs_gw_pos_v = np.stack([w2v_embedder_gw_pos_v(t) for t in tqdm(full_syn_storage_v.texts_long) ])
full_w2v_tree_gw_pos_v = KDTree(full_w2v_vecs_gw_pos_v)

In [274]:

test_v_hypos = {
    txt: 
    hypotheses_knn(
        normalize_input_text(txt), 
        #index=full_ft_tree, text2vec=ft_embedder, 
        index=full_w2v_tree_v, text2vec=w2v_embedder,
        synset_storage=full_syn_storage_v, rel_storage=rel_storage_v,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        scorer_pow=5
    )   
    for txt in tqdm(verbs_data.word)
}

In [275]:
sub = dict2submission(test_v_hypos, full_syn_storage_v.id2synset)
sub.to_csv('verbs_test_result2.tsv', sep='\t', encoding='utf-8', header=None, index=None)
sub.head(25)

,noun,result
0,hot-dog,caress.v.01
1,hot-dog,change.v.02
2,hot-dog,make.v.03
3,hot-dog,excrete.v.01
4,hot-dog,be.v.01
5,hot-dog,touch.v.01
6,hot-dog,treat.v.01
7,hot-dog,remove.v.01
8,hot-dog,interact.v.01
9,hot-dog,appear.v.02


In [276]:
! python evaluate.py data/wordnet_data/verbs_en_new.2.0-3.0.tsv verbs_test_result2.tsv

map: 0.153703018340324
mrr: 0.15460975409161934



In [283]:

test_v_hypos = {
    txt: 
    hypotheses_knn(
        normalize_input_text(txt), 
        #index=full_ft_tree, text2vec=ft_embedder, 
        index=full_w2v_tree_v_pos, text2vec=w2v_embedder_pos,
        synset_storage=full_syn_storage_v, rel_storage=rel_storage_v,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        neighbor_scorer=w2v_scorer,
        #scorer_pow=5
    )   
    for txt in tqdm(verbs_data.word)
}
sub = dict2submission(test_v_hypos, full_syn_storage_v.id2synset)
sub.to_csv('verbs_test_result_tmp.tsv', sep='\t', encoding='utf-8', header=None, index=None)
! python evaluate.py data/wordnet_data/verbs_en_new.2.0-3.0.tsv verbs_test_result_tmp.tsv


map: 0.17935479891438436
mrr: 0.18408586232420424



In [394]:
def w2v_scorer(text1, text2):
    return np.dot(w2v_embedder_gw.get_text_vec(text1), w2v_embedder_gw.get_text_vec(text2)) ** 5

test_v_hypos = {
    txt: 
    hypotheses_knn(
        normalize_input_text(txt), 
        #index=full_ft_tree, text2vec=ft_embedder, 
        #index=full_w2v_tree_v_pos, text2vec=w2v_embedder_pos_v,
        index=full_w2v_tree_v_GW, text2vec=w2v_embedder_gw,
        #index=full_w2v_tree_gw_pos_v, text2vec=w2v_embedder_gw_pos_v,
        synset_storage=full_syn_storage_v, rel_storage=rel_storage_v,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        neighbor_scorer=w2v_scorer,
        #scorer_pow=5
    )   
    for txt in tqdm(verbs_data.word)
}
sub = dict2submission(test_v_hypos, full_syn_storage_v.id2synset)
sub.to_csv('verbs_test_result_tmp.tsv', sep='\t', encoding='utf-8', header=None, index=None)
! python evaluate.py data/wordnet_data/verbs_en_new.2.0-3.0.tsv verbs_test_result_tmp.tsv


map: 0.22350316637881418
mrr: 0.2322251007484169



### debug

In [261]:
txt = normalize_input_text('Greco-Roman_wrestling')
print(txt)

greco roman wrestling


In [306]:
verbs_data.sample(3)

,word,hypernyms
11,expense,"[devalue.v.02, depreciate.v.02]"
154,eyewitness,"[witness.v.01, watch.v.01]"
5,fink,"[declare.v.04, admit.v.01]"


In [301]:
hypotheses_knn(
        'google', 
        #index=full_ft_tree, text2vec=ft_embedder, 
        index=full_w2v_tree_v_pos, text2vec=w2v_embedder_pos,
        synset_storage=full_syn_storage_v, rel_storage=rel_storage_v,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        neighbor_scorer=w2v_scorer,
        #scorer_pow=5
        verbose=True,
    )  

0.7399135323148253 1 raise.v.02 get_up 1 0.6669221426877154 0.007510991575217954
0.7399135323148253 1 swing.v.11 get_around 1 0.6669221426877154 0.007510991575217954
0.7399135323148253 1 take.v.04 get_hold_of 0 0.6669221426877154 0.007510991575217954
0.7399135323148253 1 get_off.v.02 get_off 0 0.6669221426877154 0.007510991575217954
0.7399135323148253 1 cope.v.01 get_by 1 0.6669221426877154 0.007510991575217954
0.7399135323148253 1 do.v.04 get_along 1 0.6669221426877154 0.007510991575217954
0.7399135323148253 1 arise.v.03 get_up 1 0.6669221426877154 0.007510991575217954
0.7399135323148253 1 click.v.07 get_through 0 0.6669221426877154 0.007510991575217954
0.7399135323148253 1 pull_in.v.03 get_in 1 0.6669221426877154 0.007510991575217954
0.7399135323148253 1 escape.v.01 get_away 1 0.6669221426877154 0.007510991575217954
0.7399135323148253 1 get_well.v.01 get_well 1 0.6669221426877154 0.007510991575217954
0.7399135323148253 1 get_even.v.02 get_even 1 0.6669221426877154 0.00751099157521795

['relate.v.05',
 'interact.v.01',
 'move.v.02',
 'change_state.v.01',
 'get_the_picture.v.01',
 'move.v.03',
 'act.v.01',
 'change.v.01',
 'understand.v.01',
 'better.v.03']